In [1]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ffn
from datetime import datetime, timedelta
from scipy.stats import pearsonr
import vectorbt as vbt
import scipy.stats as stats
from tapy import Indicators
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [2]:
start = "2016-01-01 UTC"
end = "2020-01-01 UTC"

prices = vbt.YFData.download(
    "AAPL",
    start=start,
    end=end
).get("Close")

In [3]:
prices.vbt.rolling_split(
    n=30,
    window_len=365*2,
    set_lens = (180,),
    left_to_right=False,
    plot=True

)
    

FigureWidget({
    'data': [{'colorscale': [[0.0, '#1f77b4'], [1.0, '#1f77b4']],
              'hoverongaps': False,
              'name': '0',
              'showlegend': True,
              'showscale': False,
              'type': 'heatmap',
              'uid': '205149e8-b413-4295-af52-d6026c6002ca',
              'x': array([datetime.datetime(2016, 1, 4, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2016, 1, 5, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2016, 1, 6, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2019, 12, 27, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 12, 30, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 12, 31, 5, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15

In [4]:
(in_price, in_indexes) , (out_price, out_indexes) = prices.vbt.rolling_split(
    n=30,
    window_len=365*2,
    set_lens = (180,),
    left_to_right=False,
)

In [5]:
def simulate_all_params(price, windows, **kwargs): 
    
    fast_ma, slow_ma = vbt.MA.run_combs(
        price, windows, r=2, short_names=["fast", "slow"]
    )
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)


    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

def simulate_best_params(price, best_fast_windows, best_slow_windows, **kwargs):
    
    fast_ma = vbt.MA.run(price, window=best_fast_windows, per_column=True)
    slow_ma = vbt.MA.run(price, window=best_slow_windows, per_column=True)

    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)

    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

In [6]:
def get_best_index(performance, higher_better=True):
    if higher_better:
        return performance[performance.groupby('split_idx').idxmax()].index
    return performance[performance.groupby('split_idx').idxmin()].index

def get_best_params(best_index, level_name):
    return best_index.get_level_values(level_name).to_numpy()

In [7]:
in_sharpe = simulate_all_params(
    in_price, 
    np.arange(10,40), 
    direction="both", 
    freq="d"
)

In [8]:
in_sharpe.vbt.plot()

FigureWidget({
    'data': [{'name': 'sharpe_ratio',
              'showlegend': True,
              'type': 'scatter',
              'uid': '8d8a4a20-2364-4eb9-b85c-dc992f53d2e3',
              'x': [(10, 11, 0), (10, 11, 1), (10, 11, 2), ..., (38, 39, 27), (38,
                    39, 28), (38, 39, 29)],
              'y': array([ 1.48300471,  1.31567617,  1.3826048 , ..., -0.8753302 , -0.80065261,
                          -0.69531075])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [9]:
in_best_index = get_best_index(in_sharpe)

in_best_fast_windows = get_best_params(
    in_best_index,
    'fast_window'
)
in_best_slow_windows = get_best_params(
    in_best_index,
    'slow_window'
)
in_best_window_pairs = np.array(
    list(
        zip(
            in_best_fast_windows, 
            in_best_slow_windows
        )
    )
)
out_test_sharpe = simulate_best_params(
    out_price, 
    in_best_fast_windows, 
    in_best_slow_windows, 
    direction="both", 
    freq="d"
)

In [10]:
in_sample_best = in_sharpe[in_best_index].values
out_sample_test = out_test_sharpe.values

t, p = stats.ttest_ind(
    a=out_sample_test,
    b=in_sample_best,
    alternative="greater"
)

In [11]:
t,p #p has to be less that 0.05

(-1.08493490604406, 0.8587802029398506)